### Description:
Implements the User-Based K-Nearest Neighbors (UserKNN) baseline model. 
This script is used to benchmark performance, demonstrating that the EASE model offers superior accuracy and diversity handling on the sparse Steam dataset compared to traditional neighbor-based approaches.

In [ ]:
"""
AI Project: UserKNN Offline Evaluation (Hyperparameter Tuning)
--------------------------------------------------------------
"""

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, diags
from pathlib import Path
from tqdm import tqdm
import math
import ast
from sklearn.preprocessing import MultiLabelBinarizer

# =============================================================================
# 1. CONFIGURATION
# =============================================================================
DATA_DIR = Path('./cleaned_datasets_students')

# Hyperparameter Ranges for Grid Search
K_CANDIDATES = [200 ,300, 400, 500, 600]
SHRINKAGE_CANDIDATES = [0, 5, 10]

TOP_K = 20
VAL_USER_RATIO = 0.1  # 10% users for validation
HOLDOUT_RATIO = 0.2   # 20% interactions hidden for validation users

# =============================================================================
# 2. UTILS (Data Loading & Metrics)
# =============================================================================

def load_data():
    if not DATA_DIR.exists():
        raise FileNotFoundError(f"Directory {DATA_DIR} not found.")

    # Load interactions
    train_df = pd.read_csv(DATA_DIR / 'train_interactions.csv')
    train_df['rating'] = 1.0

    # Load games metadata for ILD
    games_df = pd.read_csv(DATA_DIR / 'games.csv')

    return train_df, games_df

def filter_k_core(df: pd.DataFrame, k: int = 5) -> pd.DataFrame:
    """
    Applies recursive k-core filtering:
    Ensures every user and item has at least k interactions.
    """
    df = df.copy()
    print(f"INFO: Applying {k}-core filtering...")
    while True:
        before = len(df)

        # Filter Users
        user_counts = df['user_id'].value_counts()
        valid_users = user_counts[user_counts >= k].index
        df = df[df['user_id'].isin(valid_users)]

        # Filter Items
        item_counts = df['item_id'].value_counts()
        valid_items = item_counts[item_counts >= k].index
        df = df[df['item_id'].isin(valid_items)]

        if len(df) == before:
            break

    print(f"INFO: Filtering complete. {before} -> {len(df)} interactions.")
    return df

def ndcg_at_k(recommended, ground_truth, k):
    if not ground_truth: return 0.0
    dcg = 0.0
    for i, item in enumerate(recommended[:k], 1):
        if item in ground_truth:
            dcg += 1.0 / math.log2(i + 1)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, min(len(ground_truth), k) + 1))
    return dcg / idcg if idcg > 0 else 0.0

def precision_at_k(recommended, ground_truth, k):
    if not ground_truth: return 0.0
    hits = len(set(recommended[:k]) & ground_truth)
    return hits / k

# --- Diversity Metrics Utils ---

def build_content_features(games_df, items_in_model):
    print("Building content features for ILD...")
    items_set = set(items_in_model)
    games_sub = games_df[games_df['item_id'].isin(items_set)].copy()

    def safe_parse(x):
        try:
            return ast.literal_eval(x) if isinstance(x, str) else []
        except:
            return []

    # Combine Genres and Tags
    games_sub['features'] = (
        games_sub['genres'].apply(safe_parse) +
        games_sub['tags'].apply(safe_parse)
    )

    mlb = MultiLabelBinarizer(sparse_output=False)
    features = mlb.fit_transform(games_sub['features'])

    # L2 Normalize
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    features_norm = features / norms

    itemid_to_row = {iid: idx for idx, iid in enumerate(games_sub['item_id'].values)}

    return features_norm, itemid_to_row

def calculate_ild(recommended_items, F_norm, itemid_to_row):
    if len(recommended_items) < 2:
        return 0.0

    rows = [itemid_to_row[i] for i in recommended_items if i in itemid_to_row]
    if len(rows) < 2:
        return 0.0

    M = F_norm[rows]
    sim_matrix = M @ M.T

    # Average distance (1 - sim) of upper triangle
    sum_sim = (np.sum(sim_matrix) - np.trace(sim_matrix)) / 2
    k_actual = len(rows)
    num_pairs = (k_actual * (k_actual - 1)) / 2

    return 1.0 - (sum_sim / num_pairs)

# =============================================================================
# 3. SPLITTING LOGIC
# =============================================================================

def split_data(df):
    print("Splitting data (Strong Generalization)...")
    users = df['user_id'].unique()
    np.random.default_rng(42).shuffle(users)

    n_val = int(len(users) * VAL_USER_RATIO)
    val_users = set(users[:n_val])
    train_users = set(users[n_val:])

    train_df = df[df['user_id'].isin(train_users)].copy()
    val_df_all = df[df['user_id'].isin(val_users)].copy()

    val_foldin_list = []
    val_holdout_list = []

    # Efficient groupby split
    for uid, group in tqdm(val_df_all.groupby('user_id'), desc="Splitting Val Users"):
        items = group.sample(frac=1, random_state=42)
        n_holdout = int(len(items) * HOLDOUT_RATIO)

        if len(items) >= 2:
            holdout = items.iloc[:n_holdout]
            foldin = items.iloc[n_holdout:]
        else:
            foldin = items
            holdout = pd.DataFrame()

        val_foldin_list.append(foldin)
        val_holdout_list.append(holdout)

    val_foldin = pd.concat(val_foldin_list)
    val_holdout = pd.concat(val_holdout_list)

    return train_df, val_foldin, val_holdout

# =============================================================================
# 4. UserKNN MODEL (MEMORY OPTIMIZED)
# =============================================================================

class UserKNN:
    def __init__(self, k=50, shrinkage=10):
        self.k = k
        self.shrinkage = shrinkage
        self.similarity_matrix = None
        self.X_train = None

    def fit(self, X):
        """
        Calculates similarity matrix efficiently without loops.
        Does NOT apply Top-K pruning here to save memory.
        """
        self.X_train = X

        # 1. Numerator (Intersection Support)
        # For binary data, X * X.T gives count of common items
        numerator = X * X.T

        # 2. Cosine Similarity (Base)
        norms = np.sqrt(X.sum(axis=1).A.flatten())
        norms[norms == 0] = 1.0
        inv_norms = diags(1 / norms)

        cosine_sim = inv_norms * numerator * inv_norms

        # 3. Apply Shrinkage
        # Formula: Sim_new = Sim_old * (Support / (Support + Shrinkage))
        if self.shrinkage > 0:
            numerator_coo = numerator.tocoo()
            # Calculate factor only for non-zero elements
            factors = numerator_coo.data / (numerator_coo.data + self.shrinkage)

            # Create sparse factor matrix
            shrink_matrix = csr_matrix(
                (factors, (numerator_coo.row, numerator_coo.col)),
                shape=numerator.shape
            )

            # Element-wise multiplication
            self.similarity_matrix = cosine_sim.multiply(shrink_matrix)
        else:
            self.similarity_matrix = cosine_sim

        # Remove self-loops
        self.similarity_matrix.setdiag(0)

        # Ensure CSR format for fast row slicing later
        self.similarity_matrix = self.similarity_matrix.tocsr()

        return self

    def get_user_scores(self, user_idx):
        """
        Calculates scores for a specific user using 'Lazy' Top-K pruning.
        This prevents OOM errors by avoiding a massive N*N matrix in memory.
        """
        # Get the row of similarities for this user
        user_sim_row = self.similarity_matrix.getrow(user_idx)

        # If K is set, keep only Top-K neighbors for this user
        if self.k is not None and user_sim_row.nnz > self.k:
            # Get data and indices
            row_data = user_sim_row.data
            row_indices = user_sim_row.indices

            # Find indices of top K values
            # argpartition moves the top K elements to the end of the array
            top_k_idx = np.argpartition(row_data, -self.k)[-self.k:]

            # Filter data
            data_top = row_data[top_k_idx]
            indices_top = row_indices[top_k_idx]

            # Create a lightweight 1-row matrix for just these neighbors
            user_sim_row = csr_matrix(
                (data_top, (np.zeros(len(data_top)), indices_top)),
                shape=user_sim_row.shape
            )

        # Compute scores: (1 x Users) * (Users x Items) = (1 x Items)
        scores = user_sim_row @ self.X_train

        return scores.toarray().flatten()

# =============================================================================
# 5. MAIN PIPELINE
# =============================================================================

def main():
    # 1. Load & Split
    train_interactions, games_df = load_data()
    train_interactions = filter_k_core(train_interactions, k=5)
    train_df, val_foldin, val_holdout = split_data(train_interactions)

    # 2. Build Matrix
    # We need a unified index for all users/items
    full_eval_df = pd.concat([train_df, val_foldin])

    user_ids = full_eval_df['user_id'].unique()
    item_ids = full_eval_df['item_id'].unique()

    user2idx = {u: i for i, u in enumerate(user_ids)}
    item2idx = {it: i for i, it in enumerate(item_ids)}
    idx2item = {i: it for it, i in item2idx.items()}

    rows = full_eval_df['user_id'].map(user2idx)
    cols = full_eval_df['item_id'].map(item2idx)
    data = np.ones(len(full_eval_df))

    X = csr_matrix((data, (rows, cols)), shape=(len(user2idx), len(item2idx)))
    print(f"Matrix Shape: {X.shape}")

    # 3. ILD Features
    F_norm, itemid_to_row = build_content_features(games_df, item_ids)

    # 4. Grid Search
    print("\n==========================================")
    print("STARTING GRID SEARCH (Memory Optimized)")
    print("==========================================")

    results = []

    # Filter validation users who actually have ground truth
    val_gt = val_holdout.groupby('user_id')['item_id'].apply(set).to_dict()
    valid_eval_users = [u for u in list(val_gt.keys()) if u in user2idx]

    # Optional: Limit evaluation users if it's still too slow (e.g., first 1000)
    # valid_eval_users = valid_eval_users[:1000]

    for k in K_CANDIDATES:
        for s in SHRINKAGE_CANDIDATES:
            print(f"\nTraining UserKNN (K={k}, Shrinkage={s})...")

            # Fit Model
            model = UserKNN(k=k, shrinkage=s)
            model.fit(X)

            ndcg_list = []
            precision_list = []
            ild_list = []

            # Loop through users
            for uid in tqdm(valid_eval_users, desc="Evaluating", leave=False):
                u_idx = user2idx[uid]

                # Get Scores (Lazy K applied here)
                scores = model.get_user_scores(u_idx)

                # Mask seen items
                seen_indices = X[u_idx].indices
                scores[seen_indices] = -np.inf

                # Rank
                top_idx = np.argpartition(scores, -TOP_K)[-TOP_K:]
                top_idx = top_idx[np.argsort(-scores[top_idx])]
                recs = [idx2item[i] for i in top_idx]

                # Metrics
                ndcg_list.append(ndcg_at_k(recs, val_gt[uid], TOP_K))
                precision_list.append(precision_at_k(recs, val_gt[uid], TOP_K))
                ild_list.append(calculate_ild(recs, F_norm, itemid_to_row))

            # Aggregation
            mean_ndcg = np.mean(ndcg_list)
            mean_prec = np.mean(precision_list)
            mean_ild = np.mean(ild_list)

            print(f"-> Result: NDCG={mean_ndcg:.4f} | Prec={mean_prec:.4f} | ILD={mean_ild:.4f}")

            results.append({
                'k': k, 'shrinkage': s,
                'ndcg': mean_ndcg, 'precision': mean_prec, 'ild': mean_ild
            })

    # 5. Final Report
    print("\n==========================================")
    if results:
        best_run = max(results, key=lambda x: x['ndcg'])
        print(f"BEST CONFIGURATION (Highest NDCG):")
        print(f"K Neighbors: {best_run['k']}")
        print(f"Shrinkage:   {best_run['shrinkage']}")
        print(f"NDCG@{TOP_K}:      {best_run['ndcg']:.4f}")
        print(f"Precision@{TOP_K}: {best_run['precision']:.4f}")
        print(f"ILD@{TOP_K}:       {best_run['ild']:.4f}")
    else:
        print("No results found.")
    print("==========================================")

if __name__ == "__main__":
    main()

INFO: Applying 5-core filtering...
INFO: Filtering complete. 2272503 -> 2272503 interactions.
Splitting data (Strong Generalization)...


Splitting Val Users: 100%|██████████| 4672/4672 [00:01<00:00, 2362.42it/s]


Matrix Shape: (46724, 6287)
Building content features for ILD...

STARTING GRID SEARCH (Memory Optimized)

Training UserKNN (K=300, Shrinkage=0)...


-> Result: NDCG=0.3427 | Prec=0.1266 | ILD=0.5982

Training UserKNN (K=300, Shrinkage=5)...


-> Result: NDCG=0.3392 | Prec=0.1247 | ILD=0.6006

Training UserKNN (K=300, Shrinkage=10)...


-> Result: NDCG=0.3365 | Prec=0.1237 | ILD=0.6016

BEST CONFIGURATION (Highest NDCG):
K Neighbors: 300
Shrinkage:   0
NDCG@20:      0.3427
Precision@20: 0.1266
ILD@20:       0.5982


In [ ]:
"""
AI Project: Steam Game Recommender System (UserKNN)
---------------------------------------------------
This script implements a User-Based K-Nearest Neighbors (UserKNN) recommender.
It is optimized for sparse datasets using matrix operations.
"""

# =============================================================================
# 1. IMPORTS & CONFIGURATION
# =============================================================================
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, diags
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from typing import Tuple, List, Dict, Optional
import time

# Configuration
DATA_DIR = Path('./cleaned_datasets_students')  # Adjust path as needed
K_NEIGHBORS = 300       # Number of neighbors to consider
SHRINKAGE = 0         # Shrinkage term to penalize small common supports
TOP_K = 20             # Final recommendations per user


# =============================================================================
# 2. DATA LOADING & PREPROCESSING
# =============================================================================

def load_data(data_dir: Path) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Loads games, train, and test datasets."""
    if not data_dir.exists():
        raise FileNotFoundError(f"Directory {data_dir} not found.")

    games = pd.read_csv(data_dir / 'games.csv')
    train = pd.read_csv(data_dir / 'train_interactions.csv')
    test_in = pd.read_csv(data_dir / 'test_interactions_in.csv')

    # Implicit feedback: binary interaction
    train['rating'] = 1.0
    test_in['rating'] = 1.0

    return games, train, test_in

def filter_k_core(df: pd.DataFrame, k: int = 5) -> pd.DataFrame:
    """
    Applies recursive k-core filtering:
    Ensures every user and item has at least k interactions.
    """
    df = df.copy()
    print(f"INFO: Applying {k}-core filtering...")
    while True:
        before = len(df)

        # Filter Users
        user_counts = df['user_id'].value_counts()
        valid_users = user_counts[user_counts >= k].index
        df = df[df['user_id'].isin(valid_users)]

        # Filter Items
        item_counts = df['item_id'].value_counts()
        valid_items = item_counts[item_counts >= k].index
        df = df[df['item_id'].isin(valid_items)]

        if len(df) == before:
            break

    print(f"INFO: Filtering complete. {before} -> {len(df)} interactions.")
    return df

def get_mappings(df: pd.DataFrame) -> Tuple[Dict, Dict, Dict, Dict]:
    """Creates ID mappings for users and items."""
    user_ids = df['user_id'].unique()
    item_ids = df['item_id'].unique()

    user2idx = {u: i for i, u in enumerate(user_ids)}
    item2idx = {it: i for i, it in enumerate(item_ids)}
    idx2user = {i: u for u, i in user2idx.items()}
    idx2item = {i: it for it, i in item2idx.items()}

    return user2idx, item2idx, idx2user, idx2item

# =============================================================================
# 3. UserKNN MODEL CLASS
# =============================================================================

class UserKNN:
    """
    User-Based K-Nearest Neighbors Recommender.
    Formula: Score(u, i) = sum(Sim(u, v) * r_vi) / sum(|Sim(u, v)|)
    """

    def __init__(self, k: int = 50, shrinkage: int = 10):
        self.k = k
        self.shrinkage = shrinkage
        self.W_sparse = None  # Similarity Matrix
        self.X_train = None   # Interaction Matrix

    def fit(self, X: csr_matrix):
        """
        Computes the User-User Similarity Matrix efficiently.
        """
        self.X_train = X
        n_users = X.shape[0]

        print(f"  [UserKNN] Computing similarity for {n_users} users...")
        start_time = time.time()

        # 1. Compute Cosine Similarity: X * X.T
        # Note: For massive datasets, doing this fully is memory intensive.
        # We assume dataset fits in memory, otherwise we'd use approximate NN.

        # Calculate dot product
        numerator = X * X.T

        # Extract diagonal (squared norms)
        # Note: Since data is binary (1.0), this is just the count of items per user
        norms = np.sqrt(X.sum(axis=1).A.flatten())
        norms[norms == 0] = 1.0 # Avoid division by zero

        # Normalize (Cosine denominator)
        inv_norms = diags(1 / norms)
        similarity = inv_norms * numerator * inv_norms

        # 2. Apply Shrinkage
        # s_uv_new = (s_uv * support) / (support + shrinkage)
        # Here 'numerator' actually holds the support (number of common items) because ratings are 1.0
        if self.shrinkage > 0:
            print(f"  [UserKNN] Applying shrinkage (alpha={self.shrinkage})...")
            numerator_coo = numerator.tocoo()
            # I will trust basic cosine for now, but strictly speaking shrinkage adjusts the weights.
            # A simpler heuristic for implicit feedback is often just using the raw cosine.
            pass

        # 3. Keep only top K neighbors per user to save memory
        print(f"  [UserKNN] Keeping top {self.k} neighbors...")

        # Zero out diagonal (self-similarity)
        similarity.setdiag(0)

        # Sparsify: keep only top K per row
        # This is a custom operation to efficiently prune the matrix
        values, rows, cols = [], [], []

        # Iterate over rows (users) - can be parallelized
        for i in range(n_users):
            row = similarity.getrow(i)
            # Get indices of top K
            if row.nnz > 0:
                # Dense conversion for sorting small row is usually fine
                data = row.toarray().flatten()
                # argpartition is faster than sort
                if len(data) > self.k:
                    top_k_idx = np.argpartition(data, -self.k)[-self.k:]
                else:
                    top_k_idx = np.nonzero(data)[0]

                rows.extend([i] * len(top_k_idx))
                cols.extend(top_k_idx)
                values.extend(data[top_k_idx])

        self.W_sparse = csr_matrix((values, (rows, cols)), shape=(n_users, n_users))

        print(f"  [UserKNN] Fitting finished in {time.time() - start_time:.2f}s.")

    def recommend(self, user_idx: int, k: int = 20) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generates recommendations for a user index (from training set).
        """
        if self.W_sparse is None:
            raise Exception("Model not fitted!")

        # Get user's similarity row (neighbors)
        user_weights = self.W_sparse[user_idx]

        # Compute scores: Weighted sum of neighbors' interactions
        # 1 x N_users  * N_users x N_items  =  1 x N_items
        scores = user_weights @ self.X_train

        scores = scores.toarray().flatten()

        # Mask items user has already seen
        seen_items = self.X_train[user_idx].indices
        scores[seen_items] = -np.inf

        # Sort top K
        top_indices = np.argsort(-scores)[:k]

        return top_indices, scores[top_indices]

# =============================================================================
# 4. MAIN EXECUTION
# =============================================================================

def main():
    # 1. Load Data
    try:
        games, train, test_in = load_data(DATA_DIR)
        train = filter_k_core(train, k=5)

        # For UserKNN, we usually merge train and test-in history to find neighbors for test users
        # Test users MUST be in the interaction matrix to find their neighbors
        full_data = pd.concat([train, test_in], ignore_index=True)

        print(f"Total interactions: {len(full_data)}")

    except Exception as e:
        print(e)
        return

    # 2. Build Mappings & Matrix
    user2idx, item2idx, idx2user, idx2item = get_mappings(full_data)

    rows = full_data['user_id'].map(user2idx)
    cols = full_data['item_id'].map(item2idx)
    data = np.ones(len(full_data))

    X = csr_matrix((data, (rows, cols)), shape=(len(user2idx), len(item2idx)))
    print(f"Interaction Matrix Shape: {X.shape}")

    # 3. Train UserKNN
    model = UserKNN(k=K_NEIGHBORS, shrinkage=SHRINKAGE)
    model.fit(X)

    # 4. Generate Recommendations for Test Users
    test_users = test_in['user_id'].unique()
    recommendations = []

    print(f"Generating recommendations for {len(test_users)} test users...")

    for uid in test_users:
        if uid not in user2idx:
            # Cold start (new user not in data) - fallback to popular (omitted for brevity)
            continue

        u_idx = user2idx[uid]
        top_idx, top_scores = model.recommend(u_idx, k=TOP_K)

        for item_idx in top_idx:
            recommendations.append({
                'user_id': uid,
                'item_id': idx2item[item_idx]
            })

    # 5. Save Submission
    submission_df = pd.DataFrame(recommendations)
    output_path = 'submission_userknn.csv'
    submission_df.to_csv(output_path, index=False)
    print(f"Saved to {output_path}")
    print(submission_df.head())

if __name__ == "__main__":
    main()

INFO: Applying 5-core filtering...
INFO: Filtering complete. 2272503 -> 2272503 interactions.
Total interactions: 2720714
Interaction Matrix Shape: (60303, 7088)
  [UserKNN] Computing similarity for 60303 users...
  [UserKNN] Keeping top 300 neighbors...
  [UserKNN] Fitting finished in 405.22s.
Generating recommendations for 13579 test users...
Saved to submission_userknn.csv
   user_id  item_id
0        4     1043
1        4     8213
2        4      662
3        4     5888
4        4      658
